In [40]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [47]:
##Librerias

import pandas as pd
import numpy as np
import plotly as px
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets
from sklearn import tree
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
import 0_Funciones as funciones


pd.set_option('display.max_columns', None) #muestra todas las columnas de tablas
%matplotlib inline

SyntaxError: invalid decimal literal (3346906018.py, line 29)

In [23]:
#Llamamos la base de datos procesada
df_completo=pd.read_csv("https://raw.githubusercontent.com/veronica1908/RH_Entrega1/main/dfdataN.csv")
df_completo.head()



,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,DateSurvey,JobInvolvement,PerformanceRating,SurveyDate,Attrition,retirementDate,retirementType,resignationReason
0,51.0,Travel_Rarely,Sales,6.0,2,Life Sciences,1,1,Female,1,Healthcare Representative,Married,131160.0,1.0,Y,11.0,8,0,1.0,6.0,1.0,0.0,0.0,2016-12-31,3.0,4.0,2.0,2016-12-31,3,3,2016-12-31,NaN,NaN,NaN,Others
1,31.0,Travel_Frequently,Research & Development,10.0,1,Life Sciences,1,2,Female,1,Research Scientist,Single,41890.0,0.0,Y,23.0,8,1,6.0,3.0,5.0,1.0,4.0,2016-12-31,3.0,2.0,4.0,2016-12-31,2,4,2016-12-31,Yes,2016-10-15,Resignation,Others
2,32.0,Travel_Frequently,Research & Development,17.0,4,Other,1,3,Male,4,Sales Executive,Married,193280.0,1.0,Y,15.0,8,3,5.0,2.0,5.0,0.0,3.0,2016-12-31,2.0,2.0,1.0,2016-12-31,3,3,2016-12-31,NaN,NaN,NaN,Others
3,38.0,Non-Travel,Research & Development,2.0,5,Life Sciences,1,4,Male,3,Human Resources,Married,83210.0,3.0,Y,11.0,8,3,13.0,5.0,8.0,7.0,5.0,2016-12-31,4.0,4.0,3.0,2016-12-31,2,3,2016-12-31,NaN,NaN,NaN,Others
4,32.0,Travel_Rarely,Research & Development,10.0,1,Medical,1,5,Male,1,Sales Executive,Single,23420.0,4.0,Y,12.0,8,2,9.0,2.0,6.0,0.0,4.0,2016-12-31,4.0,1.0,3.0,2016-12-31,3,3,2016-12-31,NaN,NaN,NaN,Others


In [24]:
print(df_completo.info())
#df_completo.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4300 non-null   float64
 1   BusinessTravel           4300 non-null   object 
 2   Department               4300 non-null   object 
 3   DistanceFromHome         4300 non-null   float64
 4   Education                4300 non-null   int64  
 5   EducationField           4300 non-null   object 
 6   EmployeeCount            4300 non-null   int64  
 7   EmployeeID               4300 non-null   int64  
 8   Gender                   4300 non-null   object 
 9   JobLevel                 4300 non-null   int64  
 10  JobRole                  4300 non-null   object 
 11  MaritalStatus            4300 non-null   object 
 12  MonthlyIncome            4300 non-null   float64
 13  NumCompaniesWorked       4300 non-null   float64
 14  Over18                  

In [25]:
#metricas iniciales
print("Enviroent Satisfaccion mean: ",df_completo.EnvironmentSatisfaction.mean())
print("Job Satisfaccion mean: ",df_completo.JobSatisfaction.mean())

Enviroent Satisfaccion mean:  2.723953488372093
Job Satisfaccion mean:  2.7248837209302326


In [28]:
#se corrigen datos int64 que pasan a categoricas y se elimina Employee ID.
df_completo=df_completo.astype({"JobInvolvement":object,"PerformanceRating":object,"EnvironmentSatisfaction":object,"JobSatisfaction":object,"WorkLifeBalance":object,"TrainingTimesLastYear":object,"StockOptionLevel":object,"JobLevel":object,"Education":object })
X=df_completo.drop(["EmployeeID"],axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4300 non-null   float64
 1   BusinessTravel           4300 non-null   object 
 2   Department               4300 non-null   object 
 3   DistanceFromHome         4300 non-null   float64
 4   Education                4300 non-null   object 
 5   EducationField           4300 non-null   object 
 6   EmployeeCount            4300 non-null   int64  
 7   Gender                   4300 non-null   object 
 8   JobLevel                 4300 non-null   object 
 9   JobRole                  4300 non-null   object 
 10  MaritalStatus            4300 non-null   object 
 11  MonthlyIncome            4300 non-null   float64
 12  NumCompaniesWorked       4300 non-null   float64
 13  Over18                   4300 non-null   object 
 14  PercentSalaryHike       

In [29]:
##Exportar base de datos completa
df_completo.to_csv("df_completa.csv")

In [27]:
#Separar variable objetivo Attrition y sacarla de la base principal
y = pd.DataFrame(X['Attrition']) # Variable Objetivo
X.drop(['Attrition'], axis = 1, inplace = True)
y.head()

,Attrition
0,NaN
1,Yes
2,NaN
3,NaN
4,NaN


In [31]:
# LabelEncoder del target
le = LabelEncoder()
y = le.fit_transform(y)
y = pd.DataFrame(y, columns = ['Attrition'])
print(y.value_counts(), y.shape)

Attrition
1            3605
0             695
Name: count, dtype: int64 (4300, 1)


C:\Users\cesar\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
#separar variables categoricas y codificarlas
X_object_t=pd.get_dummies(X.select_dtypes(include=['object']))

In [34]:
#Separar variables numericas float
X_float=X.select_dtypes(include=['float'])

In [35]:
#se crea base de datos de X_ final
X_final=pd.concat([X_float,X_object_t],axis=1)
X_final.head()

,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_1,Education_2,Education_3,Education_4,Education_5,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobLevel_1,JobLevel_2,JobLevel_3,JobLevel_4,JobLevel_5,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,StockOptionLevel_0,StockOptionLevel_1,StockOptionLevel_2,StockOptionLevel_3,TrainingTimesLastYear_0.0,TrainingTimesLastYear_1.0,TrainingTimesLastYear_2.0,TrainingTimesLastYear_3.0,TrainingTimesLastYear_4.0,TrainingTimesLastYear_5.0,TrainingTimesLastYear_6.0,InfoDate_2016-12-31,EnvironmentSatisfaction_1.0,EnvironmentSatisfaction_2.0,EnvironmentSatisfaction_3.0,EnvironmentSatisfaction_4.0,JobSatisfaction_1.0,JobSatisfaction_2.0,JobSatisfaction_3.0,JobSatisfaction_4.0,WorkLifeBalance_1.0,WorkLifeBalance_2.0,WorkLifeBalance_3.0,WorkLifeBalance_4.0,DateSurvey_2016-12-31,JobInvolvement_1,JobInvolvement_2,JobInvolvement_3,JobInvolvement_4,PerformanceRating_3,PerformanceRating_4,SurveyDate_2016-12-31,Attrition_Yes,retirementDate_2015-01-03,retirementDate_2015-01-15,retirementDate_2015-01-26,retirementDate_2015-01-27,retirementDate_2015-01-31,retirementDate_2015-02-05,retirementDate_2015-02-07,retirementDate_2015-02-08,retirementDate_2015-02-09,retirementDate_2015-02-23,retirementDate_2015-02-26,retirementDate_2015-02-28,retirementDate_2015-03-18,retirementDate_2015-03-29,retirementDate_2015-04-07,retirementDate_2015-04-15,retirementDate_2015-04-18,retirementDate_2015-04-25,retirementDate_2015-05-04,retirementDate_2015-05-09,retirementDate_2015-05-11,retirementDate_2015-05-14,retirementDate_2015-05-19,retirementDate_2015-05-24,retirementDate_2015-05-27,retirementDate_2015-06-01,retirementDate_2015-06-08,retirementDate_2015-06-19,retirementDate_2015-06-22,retirementDate_2015-06-26,retirementDate_2015-06-27,retirementDate_2015-06-28,retirementDate_2015-07-06,retirementDate_2015-07-14,retirementDate_2015-07-16,retirementDate_2015-07-24,retirementDate_2015-08-04,retirementDate_2015-08-07,retirementDate_2015-08-11,retirementDate_2015-08-21,retirementDate_2015-09-05,retirementDate_2015-09-07,retirementDate_2015-09-16,retirementDate_2015-09-17,retirementDate_2015-09-27,retirementDate_2015-09-28,retirementDate_2015-09-30,retirementDate_2015-10-02,retirementDate_2015-10-10,retirementDate_2015-10-17,retirementDate_2015-10-19,retirementDate_2015-10-21,retirementDate_2015-10-22,retirementDate_2015-10-24,retirementDate_2015-10-28,retirementDate_2015-11-13,retirementDate_2015-11-15,retirementDate_2015-11-19,retirementDate_2015-11-22,retirementDate_2015-12-10,retirementDate_2016-01-02,retirementDate_2016-01-03,retirementDate_2016-01-04,retirementDate_2016-01-05,retirementDate_2016-01-06,retirementDate_2016-01-07,retirementDate_2016-01-08,retirementDate_2016-01-10,retirementDate_2016-01-11,retirementDate_2016-01-13,retirementDate_2016-01-14,retirementDate_2016-01-15,retirementDate_2016-01-16,retirementDate_2016-01-17,retirementDate_2016-01-18,retirementDate_2016-01-19,retirementDate_2016-01-21,retirementDate_2016-01-22,retirementDate_2016-01-23,retirementDate_2016-01-24,retirementDate_2016-01-25,retirementDate_2016-01-26,retirementDate_2016-01-27,retirementDate_2016-01-28,retirementDate_2016-01-29,retirementDate_2016-01-30,retirementDate_2016-01-31,retirementDate_2016-02-01,retirementDate_2016-02-02,retirementDate_2016-02-04,retiremen

In [36]:
X_final.shape

(4300, 441)

In [38]:
##### METODO ALTERNATIVO ESCOGIENDO modelos SEGUN IMPORTANCIA DE VARIABLES EN LOS MODELOS
#Nombre de caracteristicas númericas
numeric_columns=list(X_final.select_dtypes('float64').columns)
#Estandarización de variables
#pipeline=ColumnTransformer([( "std_numericas",StandardScaler() ,numeric_columns)], remainder='passthrough')

#X_trans = pipeline.fit_transform(X_final)
#X_trans_db=pd.DataFrame(X_trans,columns=X_final.columns)


scaler=StandardScaler() #se selecciona el escalado o estandarizacion de variables
scaler.fit(X_final) # se ajusta a la base
X_trans=scaler.transform(X_final)#se aplica a la base
X_trans_db=pd.DataFrame(X_trans,columns=X_final.columns) #se vuelve a convertir en df pandas

In [44]:
# se nombran los modelos con hiperparametros por defecto para compararlos y ya despues se afinan solo los hiperparametros del mejor
m_ranfor=RandomForestClassifier()
m_dtree=tree.DecisionTreeClassifier()
m_lreg = linear_model.LinearRegression()
m_gbr = XGBRegressor()

In [45]:
#lista con los modelos para funcion sel_variables
modelos=list([m_ranfor,m_dtree, m_lreg, m_gbr])

#se usa funcion sel_variables para comparar los desempeños de los modelos
var_names=funciones.sel_variables(modelos,X_final,y,threshold="1.5*mean")# threshold selecciona los limites de seleccion de importancia de variables, en este caso selecciona la variables que sean dos veces el promedio
var_names.shape #ANTES TENIAMOS 74 VARIABLES

NameError: name 'funciones' is not defined